In [ ]:
import cv2

In [ ]:
cv2.__version__

In [ ]:
cap = cv2.VideoCapture(0)
fgbg = cv2.createBackgroundSubtractorMOG2()
initial_height = None
height_samples = []
max_samples = 10  # Take average of first 10 samples
jump_threshold = 20  # Minimum difference in bounding box height to consider it as a jump
max_jump_height = 0  # Track the maximum jump height

In [ ]:
while True:
    ret, frame = cap.read()
    fgmask = fgbg.apply(frame)

    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        if cv2.contourArea(contour) > 500:
            (x, y, w, h) = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # Calibration phase
            if len(height_samples) < max_samples:
                height_samples.append(h)
                if len(height_samples) == max_samples:
                    initial_height = sum(height_samples) / max_samples
            elif initial_height is not None:
                jump_height = max(0, initial_height - h - jump_threshold)

                # If current jump height exceeds the max, update the max
                if jump_height > max_jump_height:
                    max_jump_height = jump_height
                    
                cv2.putText(frame, f"Jump Height: {jump_height} units", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Display the max jump height (High Score) on the top right of the frame
    frame_width = frame.shape[1]
    cv2.putText(frame, f"High Score: {max_jump_height} units", (frame_width - 400, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    cv2.imshow('Frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()